In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pip
##-- import library
import numpy as np 
import pandas as pd 
##-- Scikit-Learn
from sklearn.model_selection import train_test_split
##-- Matplot
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg
import seaborn as sns
%matplotlib inline

from sklearn.metrics import confusion_matrix
import itertools

import pickle
sns.set(style='white', context='notebook', palette='deep')

In [1]:
##-- Updata tensorflow 1.x -->  2.x
!pip install tensorflow-gpu 
!pip install tf-nightly 

In [18]:
import tensorflow as tf
print(tf.__version__)
print(tf.keras.backend.floatx())
print(tf.test.gpu_device_name())

2.2.0-dev20200112
float32
/device:GPU:0


In [0]:
from tensorflow.keras import layers, models

In [0]:
import random 
np.random.seed(99)
random.seed(99)

In [0]:
###-- データの読みこみ --###

import numpy as np
import os

class UkiyoeDataLoader(object):
    """
    Example
    -------
    >>> ukiyoe_dl = UkiyoeDataLoader()
    >>> datapath = "./data"
    >>> train_imgs, train_lbls, validation_imgs, validation_lbls = ukiyoe_dl.load(datapath)
    """
    def __init__(self, validation_size: float):
        """
        validation_size : float
        [0., 1.]
        ratio of validation data
        """
        self._basename_list = [
        'ukiyoe-train-imgs.npz',\
        'ukiyoe-train-labels.npz'
        ]
        self.validation_size = validation_size

    def load(self, datapath: str, random_seed: int=13) -> np.ndarray:
        filenames_list = self._make_filenames(datapath)
        data_list = [np.load(filename)['arr_0'] for filename in filenames_list]

        all_imgs, all_lbls = data_list
        ##########################################
        ##########################################
        # # shuffle data
        # from sklearn.utils import shuffle
        # all_imgs, all_lbls = shuffle(all_imgs, all_lbls, random_state=20191231)
        ##########################################
        ##########################################

        # shuffle data
        np.random.seed(random_seed)
        perm_idx = np.random.permutation(len(all_imgs))
        all_imgs = all_imgs[perm_idx]
        all_lbls = all_lbls[perm_idx]

        # split train and validation
        validation_num = int(len(all_lbls)*self.validation_size)

        validation_imgs = all_imgs[:validation_num]
        validation_lbls = all_lbls[:validation_num]

        train_imgs = all_imgs[validation_num:]
        train_lbls = all_lbls[validation_num:]

        return train_imgs, train_lbls, validation_imgs, validation_lbls

    def _make_filenames(self, datapath: str) -> list:
        filenames_list = [os.path.join(datapath, basename) for basename in self._basename_list]
        return filenames_list

In [22]:
###-- 分類する数 --###
num_classes = 10

###---  読み込み  ---###
datapath = "drive/My Drive/data/"

# ###--検証用データの比率
validation_size = 0.3
###-- 全体の20%をテスト用に割り当てる
X_train, Y_train, X_test, Y_test \
= UkiyoeDataLoader(validation_size).load(datapath)
X_test = X_test.astype(np.float32)
X_test = np.multiply(X_test, 1.0 / 255.0)
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
Y_test = to_categorical(Y_test, num_classes = 10)

###--型をint --> float変換する。
X_train = X_train.astype(np.float32)
###-- convert from [0:255] => [0.0:1.0]
X_train = np.multiply(X_train, 1.0 / 255.0)

###-- Encode labels to one hot vectors (ex : 2 -> [0,0,1,0,0,0,0,0,0,0])
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
Y_train = to_categorical(Y_train, num_classes = 10)


print(X_test.shape)

(947, 224, 224, 3)


###---  Definition of Model  ---###

学習およびモデルの保存

テストデータで予測

In [23]:
###---  提出用データの読み込み  ---###
###---  Data PATH  ---###
datapath_predict = "drive/My Drive/data/"

###--データの読み込み
def load(f):
    return np.load(f)['arr_0']
X_submit = load(datapath_predict+"ukiyoe-test-imgs.npz")

###--型をint --> float変換する。
X_submit = X_submit.astype(np.float32)
###-- convert from [0:255] => [0.0:1.0]
X_submit = np.multiply(X_submit, 1.0 / 255.0)

print(X_submit.shape)

(397, 224, 224, 3)


In [0]:
# ###-- 学習済みモデルの読み込み --###
###---  Data PATH  ---###
datapath_model = "drive/My Drive/save_model/"
# 重みとオプティマイザを含む全く同じモデルを再作成
model1 = tf.keras.models.load_model(datapath_model+'model1.h5')
model2 = tf.keras.models.load_model(datapath_model+'model2.h5')
model3 = tf.keras.models.load_model(datapath_model+'model3.h5')
model4 = tf.keras.models.load_model(datapath_model+'model4.h5')
model5 = tf.keras.models.load_model(datapath_model+'model5.h5')

In [0]:
###---  Validation  ---###
tem1 = model1.predict(X_test)
tem2 = model2.predict(X_test)
tem3 = model3.predict(X_test)
tem4 = model4.predict(X_test)
tem5 = model5.predict(X_test)
tem = tem1 + tem2 + tem3 + tem4 + tem5
predicts = np.argmax( tem , axis=1)
predicts = predicts.astype(np.int64)

In [27]:
score1 = model1.evaluate(X_test, Y_test, verbose=0)
score2 = model2.evaluate(X_test, Y_test, verbose=0)
score3 = model3.evaluate(X_test, Y_test, verbose=0)
score4 = model4.evaluate(X_test, Y_test, verbose=0)
score5 = model5.evaluate(X_test, Y_test, verbose=0)

print(score1[1], score2[1], score3[1], score4[1], score5[1])

0.9398099 0.9260824 0.95987326 0.9260824 0.94720167


In [28]:
score1 = model1.evaluate(X_train, Y_train, verbose=0)
score2 = model2.evaluate(X_train, Y_train, verbose=0)
score3 = model3.evaluate(X_train, Y_train, verbose=0)
score4 = model4.evaluate(X_train, Y_train, verbose=0)
score5 = model5.evaluate(X_train, Y_train, verbose=0)

print(score1[1], score2[1], score3[1], score4[1], score5[1])

0.9402985 0.93577564 0.93351424 0.91090006 0.9384894
